In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
#from tflearn.layers.core import input_data
#from tflearn.data_augmentation import ImageAugmentation

In [0]:
### neural layer # not used
def neuron_layer(x, n_neurons, name, activation = None):
    with tf.name_scope(name):
        n_inputs = int(x.get_shape()[1])
        stddev = 2/ np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev = stddev)
        w = tf.Variable(init, name = "weights")
        b = tf.Variable(tf.zeros([n_neurons], tf.float32))
        z = tf.add(tf.matmul(x,w),b)
        if activation == 'relu':
            return tf.nn.relu(z)
        elif activation == 'softmax': 
            return tf.nn.softmax(z)
        else:
            return z
        
### Loading in and preprocessing the data
def load_prep(path, testortrain = 'train' ):
    data = pd.read_csv(path)
    
    from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
    if testortrain == 'train':
        features = np.array(data.drop('label', axis = 1))
        labels = data.label.values
        features = StandardScaler().fit_transform(np.float32(features))
        return features, labels
    elif testortrain == 'test':
        features = np.array(data)
        features = StandardScaler().fit_transform(np.float32(features))
        return features
    else:
        print("-Test or Train ?")
        return None

def leaky_relu(z, name = None):
    return tf.maximum(0.01* z, z, name = name)

def generator(features, labels, batch_size):
    for i in range(0, len(features), batch_size):
        yield (features[i:i+batch_size], labels[i:i+batch_size])
        
def max_norm_regularizer(threshold = 1.0, axes = 1, name ='max_norm', collection = 'max_norm'):
    def max_norm(weights):
        clipped_weights = tf.clip_by_norm(weights, clip_norm = threshold, axes = axes)
        clip_weights = tf.assign(weights, clipped_weights, name =name)
        tf.add_to_collection(collection, clip_weights)
        return None
    return max_norm

In [3]:
from sklearn.model_selection import train_test_split

features, labels = load_prep(path ='train.csv', testortrain = 'train')
xtrain, xtest, ytrain, ytest = train_test_split(features, labels, test_size = .25,
                                                shuffle = True, stratify = labels,
                                                random_state = 42)

n_inputs = features.shape[1]
n_hidden1 = 300
n_hidden2 = 100
n_hidden3 = 50
n_outputs = 10

FileNotFoundError: ignored

In [4]:
from tensorflow.contrib.layers import batch_norm, dropout

x = tf.placeholder(tf.float32, shape = [ None, n_inputs], name = "x")
yt = tf.placeholder(tf.int64, shape = (None), name = "ytrue")

is_training = tf.placeholder(tf.bool, shape = (), name ='is_training')
keep_prob = 0.8
x_drop = dropout(x, keep_prob, is_training = is_training )
    
from tensorflow.contrib.layers import fully_connected
with tf.name_scope('dnn'):
    np.random.seed()
    he_init = tf.contrib.layers.variance_scaling_initializer()
    
    hidden1 = fully_connected(x_drop, n_hidden1, weights_initializer = he_init, scope = 'h1',
                              activation_fn  = tf.nn.elu)
    hidden1_drop = dropout(hidden1, keep_prob, is_training = is_training)
    
    hidden2 = fully_connected(hidden1_drop, n_hidden2, 
                              weights_initializer = he_init, scope = 'h2',
                              activation_fn = tf.nn.elu)
    hidden2_drop = dropout(hidden2, keep_prob, is_training = is_training)
    
    hidden3 = fully_connected(hidden2, n_hidden3, weights_initializer = he_init, scope = 'h3',
                             activation_fn = tf.nn.elu)
    hidden3_drop = dropout(hidden3, keep_prob, is_training = is_training)
    
    logits = fully_connected(hidden3_drop, n_outputs,weights_initializer = he_init,
                             scope = 'outputs', activation_fn = None)

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = yt, logits = logits)
    lossb = tf.reduce_mean(xentropy, name = 'avg_xentropy')
    reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([lossb]+ reg_loss, name = 'loss')
    
learning_rate = 0.001
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate )
    traing_op = optimizer.minimize(loss)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, yt, 1)### Chooses the class with the top prediction score
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()


NameError: ignored

In [0]:
###clip_all_weights = tf.get_collection('max_norm')
epochs = 80
batch_size = 50
with tf.Session() as sess:
    init.run()
    for epoch in range(epochs):
        train_gen = generator(xtrain, ytrain, batch_size )
        for i in range(len(xtrain)//batch_size):
            x_batch, y_batch = next(train_gen)
            sess.run(traing_op, feed_dict = {is_training: True, x: x_batch, yt: y_batch})
            ###sess.run(clip_all_weights)
        acc_train = accuracy.eval(feed_dict = {is_training: False, x: xtrain, yt: ytrain })
        acc_test = accuracy.eval(feed_dict = {is_training: False, x: xtest, yt: ytest })
        acc_dif = acc_train - acc_test
        
        print("-Epoch: ",epoch, "Training Accuracy: ", acc_train, "Test Accuracy: ",
              acc_test, "Difference: ", acc_dif)
        
        save_path = saver.save(sess, "Models/my_model.ckpt")

In [0]:
with tf.Session() as sess:
    saver.restore(sess, 'Models/my_model.ckpt')
    z = logits.eval(feed_dict ={x: load_prep(path ='test.csv', testortrain = 'test'),
                                is_training: False})
    pred = np.argmax(z, axis = 1)

In [0]:
predictions = pd.DataFrame(pred, index = np.arange(1, len(pred)+1), columns = ['Label'])

In [0]:
predictions.to_csv('predictions/nn_pred.csv', index_label = 'ImageId')